In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellem.musique.qa import answer_question_standard
from bellem.utils import set_seed, jprint
from bellem.musique.singlehop import benchmark

set_seed(89)

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
pd.options.display.float_format = '{:,.3f}'.format

In [5]:
from bellem.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)

print(len(df))
df.head()

100


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar]
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County]
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River]
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith]
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster]


In [6]:
df['paragraphs'] = df['paragraphs'].map(lambda ps: [p for p in ps if p['is_supporting']])

## Definitions

In [7]:
perfect_retrieval_func = lambda docs, query: [doc for doc in docs if doc['is_supporting']]

In [8]:
qa_func = answer_question_standard

## Experiments

In [9]:
results = []

In [10]:
completion_params = {
    "temperature": 0.1
}

### llama-sft

In [11]:
def replace_paragraphs_with_offline_triplets(df, jerx_file):
    df = df.copy()

    jerx_df = pd.read_json(jerx_file, lines=True)
    jerx_mapping = {(row['id'], row['paragraph_idx']): row['generation'] for _, row in jerx_df.iterrows()}

    def replace_paragraphs(row):
        new_paragraphs = []
        for p in row['paragraphs']:
            p = deepcopy(p) 
            triplets_str = jerx_mapping[(row['id'], p['idx'])].strip()
            p['paragraph_text'] = triplets_str
            new_paragraphs.append(p)
        row['paragraphs'] = new_paragraphs
        return row

    return df.progress_apply(replace_paragraphs, axis=1) 

In [12]:
# df_llama_sft = replace_paragraphs_with_offline_triplets(df, Path("../../data/raw/musique-evaluation/jerx-inferences/llama3-sft-aw7ihmbc-ablation.jsonl"))
# df_llama_sft.head()

In [13]:
# for i in range(1,4):
#     df_llama_sft, scores = benchmark(df_llama_sft, qa_func, perfect_retrieval_func)
#     results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-sft", "run": i})
#     jprint(scores)

### llama-zero-shot

In [28]:
from bellem.jerx.fewshot.llm import make_kg_triplet_extract_fn, DEFAULT_JERX_SYSTEM_MESSAGE_FOR_LLAMA

prefix_messages = [
    dict(role="system", content=DEFAULT_JERX_SYSTEM_MESSAGE_FOR_LLAMA),
]
extract_kg_triplets = make_kg_triplet_extract_fn(model='llama3-8b-togetherai', prefix_messages=prefix_messages, completion_params=completion_params)

def replace_paragraphs(row):
    new_paragraphs = []
    for p in row['paragraphs']:
        p = deepcopy(p) 
        triplets_str = '\n'.join(" | ".join(triplet) for triplet in extract_kg_triplets(p['paragraph_text']))
        p['paragraph_text'] = '\n'.join([triplets_str])
        new_paragraphs.append(p)
    row['paragraphs'] = new_paragraphs
    return row

In [29]:
for i in range(1,4):
    df_llama_zs = df.progress_apply(replace_paragraphs, axis=1) 
    df_llama_zs, scores = benchmark(df_llama_zs, qa_func, perfect_retrieval_func)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-zero-shot", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.47,
  "f1": 0.5637912087912087,
  "fuzzy_match": 0.59
}


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.54,
  "f1": 0.6247142857142856,
  "fuzzy_match": 0.65
}


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.49,
  "f1": 0.6143809523809522,
  "fuzzy_match": 0.66
}


### llama-few-shot

In [16]:
from bellem.jerx.fewshot.llm import make_kg_triplet_extract_fn

extract_kg_triplets = make_kg_triplet_extract_fn(model='llama3-8b-togetherai', completion_params=completion_params)

def replace_paragraphs(row):
    new_paragraphs = []
    for p in row['paragraphs']:
        p = deepcopy(p) 
        triplets_str = '\n'.join(" | ".join(triplet) for triplet in extract_kg_triplets(p['paragraph_text']))
        p['paragraph_text'] = '\n'.join([triplets_str])
        new_paragraphs.append(p)
    row['paragraphs'] = new_paragraphs
    return row

  0%|          | 0/100 [00:00<?, ?it/s]

Amanollah Khan Zia' os-Soltan | occupation | Politician
Amanollah Khan Zia' os-Soltan | occupation | Aristocrat
Amanollah Khan Zia' os-Soltan | affiliation | Qajar court


In [17]:
for i in range(1,4):
    df_llama_fs = df.progress_apply(replace_paragraphs, axis=1) 
    df_llama_fs, scores = benchmark(df_llama_fs, qa_func, perfect_retrieval_func)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-few-shot", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.56,
  "f1": 0.6545581085581085,
  "fuzzy_match": 0.71
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.56,
  "f1": 0.6632723942723944,
  "fuzzy_match": 0.69
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.57,
  "f1": 0.6658914418914419,
  "fuzzy_match": 0.7
}


## Report

In [30]:
with open('ablation-jerx-llama-shqa-results.jsonl', 'w') as f:
    for result in results:
        f.write(json.dumps(result))
        f.write('\n')

In [22]:
report_df = pd.read_json('ablation-jerx-llama-shqa-results.jsonl', lines=True).loc[:, ['jerx', 'run', 'exact_match', 'f1']].copy()
report_df.rename(columns={'jerx': 'Model', 'exact_match': 'EM', 'f1': 'F1'}, inplace=True)
model_name_map = {
    'llama-zero-shot': 'Zero-shot prompted',
    'llama-few-shot': 'Few-shot prompted',
    'llama-sft': 'Supervised fine-tuned',
}
report_df['Model'] = report_df['Model'].map(lambda x: model_name_map[x])

In [23]:
with open("ablation-jerx-llama-mhqa-results-all.tex", 'w') as f:
    f.write(report_df.to_latex(index=False, float_format='%.3f'))

In [25]:
agg_report_df = report_df[['Model', 'EM', 'F1']].groupby('Model').mean().loc[['Zero-shot prompted', 'Few-shot prompted', 'Supervised fine-tuned']].reset_index()
agg_report_df

,Model,EM,F1
0,Zero-shot prompted,0.500,0.593
1,Few-shot prompted,0.563,0.661
2,Supervised fine-tuned,0.557,0.646


In [26]:
with open("ablation-jerx-llama-mhqa-results-agg.tex", 'w') as f:
    f.write(agg_report_df.to_latex(index=False, float_format='%.3f'))